In [122]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.externals import joblib

In [2]:
df = pd.read_pickle('song_comps.pkl')

In [3]:
df = df.dropna()

In [4]:
def normalize(data):
    if data.name  in ['mode', 'genre', 'Non-Standard Time Signature']:
        return data
    return (data - data.mean()) / data.std()
df = df.apply(normalize)

In [5]:
X = df.drop('genre', axis=1).values
Y = df.genre.values

In [65]:
ind = np.random.permutation(len(Y))

In [66]:
sub_X = X[ind[:25000], :]
sub_Y = Y[ind[:25000]]

In [67]:
test_X = X[ind[25000:30000], :]
test_Y = Y[ind[25000:30000]]

In [68]:
test_Y.mean()

0.21060000000000001

In [89]:
model = SVC(probability=True)

In [90]:
model.fit(sub_X, sub_Y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.0,
  kernel='rbf', max_iter=-1, probability=True, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [91]:
model.score(test_X, test_Y)

0.91579999999999995

Model classifies correctly 91.5% of the time

In [78]:
predictions = model.predict(test_X)

In [120]:
model.predict_proba(test_X[(predictions != test_Y), :]).max(axis=1).mean()

0.73726398578976204

In [121]:
model.predict_proba(test_X[(predictions == test_Y), :]).max(axis=1).mean()

0.92919430977101558

Average probability is much higher for correct predictions than incorrect (i.e. the model is more sure about the correct predictions than the incorrect which matches intuition)

In [123]:
joblib.dump(model, 'model.pkl')

['model.pkl',
 'model.pkl_01.npy',
 'model.pkl_02.npy',
 'model.pkl_03.npy',
 'model.pkl_04.npy',
 'model.pkl_05.npy',
 'model.pkl_06.npy',
 'model.pkl_07.npy',
 'model.pkl_08.npy',
 'model.pkl_09.npy',
 'model.pkl_10.npy',
 'model.pkl_11.npy']